In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
import lightgbm as lgb

import networkx as nx
from networkit import linkprediction as lp, nxadapter
import seaborn as sns
from functools import partial

In [3]:
G = nx.read_edgelist('/Users/s0c02nj/Desktop/IT-Blog-master/scripts/link-prediction/data/graph.txt', comments='#')

In [5]:
valid_graph = nxadapter.nx2nk(G)

In [6]:
# Training and test graphs creation
test_graph = lp.RandomLinkSampler.byPercentage(valid_graph, 0.9)
train_graph = lp.RandomLinkSampler.byPercentage(test_graph, 0.7)


In [7]:
# Training and testing sets creation
testing_set = lp.MissingLinksFinder(test_graph).findAtDistance(2)
training_set = lp.MissingLinksFinder(train_graph).findAtDistance(2)


In [14]:
def concatenate(node_set, label):
    dataset = pd.DataFrame({'nodes': node_set, 'label': label})
    return dataset

In [15]:
# Label creation
def assign_label(pair, graph):
    u, v = pair[0], pair[1]
    return (int(graph.hasEdge(u, v)))


In [12]:
y_train = list(map(partial(assign_label, graph=test_graph), training_set))
y_test = list(map(partial(assign_label, graph=valid_graph), testing_set))

In [16]:
# Concatenation of labels with samples
train = concatenate(training_set, y_train)
test = concatenate(testing_set, y_test)
trainingSet = train.nodes.values
testingSet = test.nodes.values

In [22]:
# Feature engineering
    trainLPs = [
        lp.CommonNeighborsIndex(train_graph), lp.JaccardIndex(train_graph),
        lp.AdamicAdarIndex(train_graph), lp.ResourceAllocationIndex(train_graph),
        lp.PreferentialAttachmentIndex(train_graph), lp.AdjustedRandIndex(train_graph),
        lp.NeighborhoodDistanceIndex(train_graph), lp.TotalNeighborsIndex(train_graph),
        lp.SameCommunityIndex(train_graph), lp.UDegreeIndex(train_graph),
        lp.VDegreeIndex(train_graph)
    ]

In [34]:
#X_train = lp.getFeatures(trainingSet, *trainLPs)